In [65]:
%load_ext watermark
%watermark -a Filippo_Valle -p pandas,numpy,matplotlib,seaborn -m -r -g

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Filippo_Valle

pandas    : 1.3.3
numpy     : 1.20.3
matplotlib: 3.4.3
seaborn   : 0.11.2

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.11.0-36-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: 08bdb1223c3304a6222c0e5826a5958072b1e2be

Git repo: git@github.com:fvalle1/keywordTCGA.git



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_clusters = pd.read_excel("https://github.com/BioPhys-Turin/Supplementary_miRNA_cluster/blob/master/conversion_file.xlsx?raw=true").set_index("MiRNA cluster ID").drop("# miRNAs", axis=1)

In [ ]:
df_topics = pd.read_csv("brca/trisbm/trisbm_level_0_metadata.csv")

In [ ]:
df_clusters

In [ ]:
df_mirnas = pd.DataFrame(data=df_topics.values.ravel()).dropna().set_index(0)
df_mirnas["cluster"]=""
df_mirnas["topic"]=""
df_mirnas["count"]=1

In [ ]:
df_cluster_assignment = pd.DataFrame()
for cluster in df_clusters.index:
    data = list(map(lambda mir: mir.replace("miR","mir"),df_clusters.loc[cluster,:].dropna().values))
    for mirna in data:
        if mirna in df_mirnas.index:
            df_mirnas.at[mirna,"cluster"]=cluster

In [ ]:
for topic in df_topics.columns:
    data = df_topics[topic].dropna().values
    for mirna in data:
        if mirna in df_mirnas.index:
            df_mirnas.at[mirna,"topic"]=topic

In [ ]:
df_mirnas = df_mirnas[df_mirnas["cluster"]!=""]

In [ ]:
df_mirnas.to_csv("cantini2019/files.dat")

In [ ]:
df_over = df_mirnas.groupby(["topic","cluster"]).count()
df_over[df_over["count"]>1]

In [ ]:
df_mirnas[(df_mirnas["topic"]=="Metadatum 7") & (df_mirnas["cluster"]=="cl349_chr14")].index

In [ ]:
from topicpy.hypergeom import parameters_for_hypergeometric, build_map, plot_map

In [ ]:
df_pvals = build_map(*parameters_for_hypergeometric(df_mirnas["topic"], df_mirnas["cluster"]), True)
#.applymap(lambda log: np.log2(10)*log)
df_pvals.index = df_mirnas["topic"].unique()
df_pvals.columns = df_mirnas["cluster"].unique()

In [ ]:
df_pvals.loc["Metadatum 7", "cl349_chr14"], df_pvals.loc["Metadatum 11", "cl590_chr19"]

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score

In [ ]:
adjusted_mutual_info_score(df_mirnas["topic"].values, df_mirnas["cluster"].values)

In [ ]:
network_pal = sns.color_palette('husl', n_colors=len(df_pvals.columns))

# Create a dictionary where the key is the category and the values are the
# colors from the palette we just created
network_lut = dict(zip(df_pvals.columns, network_pal))
network_col = df_pvals.columns.map(network_lut)
# Create a dictionary where the key is the category and the values are the
# colors from the palette we just created
network_lut = dict(zip(df_pvals.columns, network_pal))
network_col = df_pvals.columns.map(network_lut)

cm = sns.clustermap(df_pvals,
                    row_cluster=False,
                    col_cluster=False,
                    metric='euclidean',
                    vmin=0,
                    vmax=30,
                    cmap='Blues_r',
                    col_colors=network_col,
                    mask=False,
                    cbar_pos=(1.05, 0.05, 0.05, 0.7))

ax = cm.ax_heatmap
ax.tick_params(labelsize=15)

ax.set_ylabel("miRNA-topic", fontsize=35)
ax.set_xlabel("cluster", fontsize=35)
ax.set_xticklabels(df_pvals.columns, rotation=75)
ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.set_yticklabels(df_pvals.index, rotation=0)

cax = cm.ax_cbar
cax.tick_params(labelsize=35)
cax.set_title("-Log2(P-value)", fontsize=30)

# plt.tight_layout()

#cm.fig.suptitle('Algorithm comparison', fontsize=40)
#cm.savefig(f"topics_logp_{first_name}_{last_name}.pdf")
plt.show()

In [ ]:
#from topicpy.hsbmpy import clusteranalysis
#clusteranalysis("cantini2019/", ["cluster"], "trisbm")